In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-16 05:35:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.51MB/s    in 0.2s    

2020-11-16 05:35:06 (5.51 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import pyspark.sql.functions as F

# load in a sql function to access the columns 
from pyspark.sql.functions import col

In [ ]:
# DELIVERABLE 2!!!!  Determine Bias of Vine Reviews

In [7]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz-db.clviqkfcmhow.us-east-2.rds.amazonaws.com:5432/challenge-db"
config = {"user":"postgres", 
          "password": "01eellv01", 
          "driver":"org.postgresql.Driver"}

In [8]:
cleaned_vine_info = spark.read.jdbc(url= jdbc_url, table= "vine_table", properties=config)
cleaned_vine_info.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [9]:
# step 1) vine_df() total_votes >= 20 
cleaned_vine_df = cleaned_vine_info.filter(col("total_votes") >= 20)
cleaned_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R2M1TGWAI7ED73|          5|          110|        142|   N|                N|
| REC11EG59BRCQ|          1|           13|         90|   N|                N|
|R2LKLC82XA1LIM|          1|           18|         35|   N|                N|
|R16UMKX9GH7UVH|          5|           26|         28|   N|                N|
|R18DZ5ZM2YB4W7|          1|            8|        145|   N|                N|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
| RK5JE7SGNTL35|          3|           19|         55|   N|     

In [10]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
# change the strings to ints... helpful_votes,
cleaned_vine_df = cleaned_vine_df.withColumn("helpful_votes", col("helpful_votes").cast("Integer"))


In [11]:

vote_vine_info = cleaned_vine_df.withColumn("div_votes", cleaned_vine_df["helpful_votes"]/cleaned_vine_df["total_votes"])
vote_vine_info.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          div_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N| 0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                0.6|
|R2M1TGWAI7ED73|          5|          110|        142|   N|                N| 0.7746478873239436|
| REC11EG59BRCQ|          1|           13|         90|   N|                N|0.14444444444444443|
|R2LKLC82XA1LIM|          1|           18|         35|   N|                N| 0.5142857142857142|
|R16UMKX9GH7UVH|          5|           26|         28|   N|                N| 0.9285714285714286|
|R18DZ5ZM2YB4W7|          1|            8|        145|   N|                N|0.05517241379310345|
|R127WEQY2FM1T3|    

In [12]:
# step 2) Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all 
# the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

vote_vine_df =vote_vine_info.filter(col("div_votes") >= 0.5)
vote_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         div_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|               0.6|
|R2M1TGWAI7ED73|          5|          110|        142|   N|                N|0.7746478873239436|
|R2LKLC82XA1LIM|          1|           18|         35|   N|                N|0.5142857142857142|
|R16UMKX9GH7UVH|          5|           26|         28|   N|                N|0.9285714285714286|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|              0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|0.9166666666666666|
|R1U3AR67RE273L|          1|  

In [13]:
# Step 3) Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows 
# where a review was written as part of the Vine program (paid), vine == 'Y'.

written_review_df = vote_vine_df.filter(col("vine") == "Y")
written_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         div_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1OWK33OPI45KT|          5|           55|         58|   Y|                N|0.9482758620689655|
|R2UP7VTED8O425|          5|           35|         41|   Y|                N|0.8536585365853658|
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|0.8888888888888888|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|               1.0|
|R3N4QAXTQCA1ED|          5|           18|         20|   Y|                N|               0.9|
| ROXZOMHBY8Y9S|          4|           21|         23|   Y|                N|0.9130434782608695|
|R3EPHK0JGQGE91|          5|           21|         28|   Y|                N|              0.75|
| RCP3KRO47S0GG|          4|  

In [14]:
# Step 4) Same as step 3, but we want a (seperate?) dataframe for all the vines WITHOUT a written review.
no_written_review_df = vote_vine_df.filter(col("vine")== "N")
no_written_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         div_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|               0.6|
|R2M1TGWAI7ED73|          5|          110|        142|   N|                N|0.7746478873239436|
|R2LKLC82XA1LIM|          1|           18|         35|   N|                N|0.5142857142857142|
|R16UMKX9GH7UVH|          5|           26|         28|   N|                N|0.9285714285714286|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|              0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|0.9166666666666666|
|R1U3AR67RE273L|          1|  

In [ ]:
# Step 5) Determine the total number of reviews, 
# the number of 5-star reviews, 
# and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [15]:
# Step 5 work part 1 | total number of reviews (40,565)
vote_vine_df.count()

40565

In [16]:
# Step 5 work part 2 | number of 5 star reviews (15,711)
five_star_df = vote_vine_df.filter(col("star_rating") == 5)
five_star_df.count()


15711

In [18]:
# Step 5 work part 3 | **PAID** |  written_review_df % of 5 star reviews | 94 total reviews | 48 5 star reviews  | 51% (5) star reviews....
# written_review_df.count()
five_star_written_df = written_review_df.filter(col("star_rating") == 5)
five_star_written_df.count()

48

In [20]:
# Step 5 work park 4 | **UNPAID** | no_written_review_df 5 star reviews | 40,471 (non-paid) total reviews | 15,663 (5) star reviews | 38.7% (5) star

# no_written_review_df.count()

unpaid_five_star_df = no_written_review_df.filter(col("star_rating") == 5)
unpaid_five_star_df.count()

40471